In [1]:
# Module 0: Setup (Colab + Imports)

# Do NOT pin an older TF version; just ensure it's installed.
!pip install -q tensorflow matplotlib tqdm

import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses

from tqdm import tqdm

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0


In [2]:
# Config (you can later move this to a YAML file and parse it)
CONFIG = {
    "image_size": 28,          # MNIST
    "channels": 1,             # grayscale
    "latent_dim": 100,
    "batch_size": 64,
    "buffer_size": 60000,
    "epochs": 100,
    "lr": 0.0002,
    "beta1": 0.5,
    "train_split": 0.95,
    "normalization": "tanh",
    "samples_dir": "samples",
    "checkpoints_dir": "checkpoints"
}

os.makedirs(CONFIG["samples_dir"], exist_ok=True)
os.makedirs(CONFIG["checkpoints_dir"], exist_ok=True)


In [3]:
# Data loader (MNIST as example domain)
def load_mnist_dataset(config=CONFIG):
    (x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()

    x = np.concatenate([x_train, x_test], axis=0)  # 70k images
    x = x.astype("float32")

    # Add channel dimension
    x = np.expand_dims(x, axis=-1)  # (N, 28, 28, 1)

    # Resize if needed
    if config["image_size"] != 28:
        x = tf.image.resize(x, (config["image_size"], config["image_size"])).numpy()

    # Normalization to [-1, 1] for Tanh output
    x = (x / 127.5) - 1.0

    # Train / test split (for evaluation only)
    n = x.shape[0]
    n_train = int(config["train_split"] * n)
    x_train = x[:n_train]
    x_val = x[n_train:]

    # Create tf.data pipeline
    train_ds = tf.data.Dataset.from_tensor_slices(x_train)\
        .shuffle(config["buffer_size"])\
        .batch(config["batch_size"], drop_remainder=True)

    val_ds = tf.data.Dataset.from_tensor_slices(x_val)\
        .batch(config["batch_size"], drop_remainder=True)

    return train_ds, val_ds

train_ds, val_ds = load_mnist_dataset(CONFIG)
len(list(train_ds)), len(list(val_ds))


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


(1039, 54)

In [4]:
def build_generator(config=CONFIG):
    latent_dim = config["latent_dim"]
    img_size = config["image_size"]
    channels = config["channels"]
    img_shape = (img_size, img_size, channels)

    model = models.Sequential(name="Generator")

    # Fully connected -> reshape to small feature map
    model.add(layers.Input(shape=(latent_dim,)))
    model.add(layers.Dense(7 * 7 * 256, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Reshape((7, 7, 256)))

    # Up-sampling block 1
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # Up-sampling block 2
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # Output layer
    model.add(layers.Conv2D(channels, kernel_size=3, padding="same", activation="tanh"))

    return model


def build_discriminator(config=CONFIG):
    img_size = config["image_size"]
    channels = config["channels"]

    model = models.Sequential(name="Discriminator")
    model.add(layers.Input(shape=(img_size, img_size, channels)))

    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))

    return model


generator = build_generator(CONFIG)
discriminator = build_discriminator(CONFIG)

generator.summary()
discriminator.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12544)          │     1,254,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12544)          │        50,176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 14, 14, 128)    │       524,288 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 28, 28, 64)     │       131,072 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 1)      │           577 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,961,281 (7.48 MB)

 Trainable params: 1,935,809 (7.38 MB)

...

Model: "Discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,073 (543.25 KB)

 Trainable params: 138,817 (542.25 KB)

 Non-trainable params: 256 (1.00 KB)

In [5]:
# Combined Vanilla GAN model wrapper (for organization like vanilla_gan.py)
class VanillaGAN(tf.keras.Model):
    def __init__(self, generator, discriminator, config=CONFIG):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = config["latent_dim"]

        self.d_optimizer = optimizers.Adam(config["lr"], beta_1=config["beta1"])
        self.g_optimizer = optimizers.Adam(config["lr"], beta_1=config["beta1"])
        self.loss_fn = losses.BinaryCrossentropy(from_logits=False)

        self.d_loss_metric = tf.keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = tf.keras.metrics.Mean(name="g_loss")

    def generate_noise(self, batch_size):
        return tf.random.normal(shape=(batch_size, self.latent_dim))

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]

        # -------------------
        # Train Discriminator
        # -------------------
        random_noise = self.generate_noise(batch_size)

        with tf.GradientTape() as tape:
            fake_images = self.generator(random_noise, training=True)

            # Labels: real=1, fake=0
            real_labels = tf.ones((batch_size, 1))
            fake_labels = tf.zeros((batch_size, 1))

            real_logits = self.discriminator(real_images, training=True)
            fake_logits = self.discriminator(fake_images, training=True)

            d_loss_real = self.loss_fn(real_labels, real_logits)
            d_loss_fake = self.loss_fn(fake_labels, fake_logits)
            d_loss = d_loss_real + d_loss_fake

        grads = tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_variables))

        # ---------------
        # Train Generator
        # ---------------
        random_noise = self.generate_noise(batch_size)
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_noise, training=True)
            # Generator wants discriminator to output 1 for fake images
            misleading_labels = tf.ones((batch_size, 1))
            fake_logits = self.discriminator(fake_images, training=True)
            g_loss = self.loss_fn(misleading_labels, fake_logits)

        grads = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_variables))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)

        return {"d_loss": self.d_loss_metric.result(), "g_loss": self.g_loss_metric.result()}

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]


In [6]:
# Fixed noise for monitoring progress
fixed_noise = tf.random.normal(shape=(16, CONFIG["latent_dim"]))

def save_sample_images(generator, epoch, samples_dir=CONFIG["samples_dir"]):
    preds = generator(fixed_noise, training=False)
    imgs = (preds + 1.0) / 2.0  # back to [0,1]

    fig, axes = plt.subplots(4, 4, figsize=(4, 4))
    idx = 0
    for i in range(4):
        for j in range(4):
            ax = axes[i, j]
            ax.imshow(imgs[idx, :, :, 0], cmap="gray")
            ax.axis("off")
            idx += 1
    plt.tight_layout()
    os.makedirs(samples_dir, exist_ok=True)
    out_path = os.path.join(samples_dir, f"epoch_{epoch:03d}.png")
    plt.savefig(out_path)
    plt.close(fig)
    print(f"Saved samples to {out_path}")


In [ ]:
# Training loop (manual, similar to train.py)
gan = VanillaGAN(generator, discriminator, CONFIG)

@tf.function
def train_step(images):
    return gan.train_step(images)

def train_gan(train_ds, epochs=CONFIG["epochs"]):
    for epoch in range(1, epochs + 1):
        print(f"\nEpoch {epoch}/{epochs}")
        gan.d_loss_metric.reset_state()
        gan.g_loss_metric.reset_state()

        for batch in tqdm(train_ds, leave=False):
            train_step(batch)

        d_loss = gan.d_loss_metric.result().numpy()
        g_loss = gan.g_loss_metric.result().numpy()
        print(f"D_loss: {d_loss:.4f} | G_loss: {g_loss:.4f}")

        # Save samples periodically
        if epoch % 5 == 0 or epoch == 1:
            save_sample_images(gan.generator, epoch)

        # Checkpoint (simple H5 save)
        if epoch % 10 == 0:
            g_path = os.path.join(CONFIG["checkpoints_dir"], f"G_epoch_{epoch:03d}.keras")
            d_path = os.path.join(CONFIG["checkpoints_dir"], f"D_epoch_{epoch:03d}.keras")
            gan.generator.save(g_path)
            gan.discriminator.save(d_path)
            print(f"Saved checkpoints at epoch {epoch}")

train_gan(train_ds, epochs=CONFIG["epochs"])


Epoch 1/100


D_loss: 0.9620 | G_loss: 1.1748
Saved samples to samples/epoch_001.png

Epoch 2/100


D_loss: 1.0541 | G_loss: 1.0804

Epoch 3/100


D_loss: 1.1112 | G_loss: 1.0465

Epoch 4/100


D_loss: 1.1580 | G_loss: 1.0140

Epoch 5/100


...

D_loss: 1.2524 | G_loss: 0.9247

Epoch 14/100


D_loss: 1.2553 | G_loss: 0.9214

Epoch 15/100


D_loss: 1.2573 | G_loss: 0.9111
Saved samples to samples/epoch_015.png

Epoch 16/100


D_loss: 1.2611 | G_loss: 0.9067

Epoch 17/100


 37%|███▋      | 388/1039 [00:08<00:14, 45.66it/s]

In [ ]:
g_final_path = os.path.join(CONFIG["checkpoints_dir"], "G_final.keras")
d_final_path = os.path.join(CONFIG["checkpoints_dir"], "D_final.keras")
gan.generator.save(g_final_path)
gan.discriminator.save(d_final_path)
print("Saved final models.")


In [ ]:
# Visualize a grid of generated images
def show_generated_grid(generator, n=25):
    noise = tf.random.normal((n, CONFIG["latent_dim"]))
    preds = generator(noise, training=False)
    imgs = (preds + 1.0) / 2.0

    side = int(np.sqrt(n))
    fig, axes = plt.subplots(side, side, figsize=(side, side))
    idx = 0
    for i in range(side):
        for j in range(side):
            ax = axes[i, j]
            ax.imshow(imgs[idx, :, :, 0], cmap="gray")
            ax.axis("off")
            idx += 1
    plt.tight_layout()
    plt.show()

show_generated_grid(gan.generator, n=25)


In [ ]:
# Simple diversity score: measure variance across generated images
def compute_diversity_score(generator, num_samples=1000):
    noise = tf.random.normal((num_samples, CONFIG["latent_dim"]))
    preds = generator(noise, training=False).numpy()
    preds = (preds + 1.0) / 2.0

    # Flatten images and compute variance across the dataset
    flat = preds.reshape(num_samples, -1)
    variance_per_pixel = flat.var(axis=0)
    diversity_score = variance_per_pixel.mean()
    return diversity_score

diversity = compute_diversity_score(gan.generator, num_samples=512)
print("Diversity score (higher ~ more variation):", diversity)


In [ ]:
# Train a simple classifier on real MNIST digits and compare confidence on real vs fake
def train_simple_classifier(train_ds, val_ds):
    model = models.Sequential([
        layers.Input(shape=(CONFIG["image_size"], CONFIG["image_size"], CONFIG["channels"])),
        layers.Conv2D(32, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    # Reload MNIST with labels (for simplicity)
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x = np.concatenate([x_train, x_test], axis=0).astype("float32")
    y = np.concatenate([y_train, y_test], axis=0)
    x = np.expand_dims(x, -1)
    x = (x / 127.5) - 1.0

    model.fit(x, y, batch_size=128, epochs=3, validation_split=0.1, verbose=2)
    return model

classifier = train_simple_classifier(train_ds, val_ds)


In [ ]:
# Realism score: classifier confidence on generated images
def classifier_realism_score(generator, classifier, num_samples=1000):
    noise = tf.random.normal((num_samples, CONFIG["latent_dim"]))
    fake_imgs = generator(noise, training=False)
    # Rescale to [-1,1] already; classifier expects same normalization
    preds = classifier.predict(fake_imgs, verbose=0)
    confidences = preds.max(axis=1)
    return confidences.mean()

realism = classifier_realism_score(gan.generator, classifier, num_samples=512)
print("Classifier-based realism score:", realism)


In [ ]:
# Inference utilities: generate N images and display/save
def generate_and_save(generator, n_images=16, out_dir="generated_batch"):
    os.makedirs(out_dir, exist_ok=True)
    noise = tf.random.normal((n_images, CONFIG["latent_dim"]))
    preds = generator(noise, training=False)
    imgs = (preds + 1.0) / 2.0

    for i in range(n_images):
        img = imgs[i, :, :, 0]
        plt.imsave(os.path.join(out_dir, f"img_{i:04d}.png"), img, cmap="gray")
    print(f"Saved {n_images} images to {out_dir}")

generate_and_save(gan.generator, n_images=32)


In [ ]:
# Install Streamlit and ngrok for public URL
!pip install -q streamlit pyngrok

# Download ngrok (one-time setup)
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xvzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok authtoken YOUR_NGROK_TOKEN  # Get free token from ngrok.com


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import zipfile
from datetime import datetime
import io

# Load your trained Generator (from checkpoints)
@st.cache_resource
def load_generator():
    generator = tf.keras.models.load_model("checkpoints/G_final.keras")
    return generator

# Generate images
def generate_images(generator, n_images, latent_dim=100, img_size=64, channels=1):
    noise = tf.random.normal([n_images, latent_dim])
    predictions = generator(noise, training=False)

    # Denormalize from [-1,1] to [0,1]
    predictions = (predictions + 1) / 2
    return predictions.numpy()

# Main app
st.set_page_config(page_title="Synthetic Image Generator", layout="wide")
st.title("🔬 Privacy-Preserving Synthetic Image Generator")
st.markdown("**Vanilla GAN Deployment** | Generate synthetic images for data augmentation")

# Sidebar controls
st.sidebar.header("Generation Controls")
n_images = st.sidebar.slider("Number of images", 1, 100, 16)
seed = st.sidebar.slider("Random seed", 0, 10000, 42)
img_size = st.sidebar.selectbox("Image size", [28, 64, 128])
batch_size = st.sidebar.slider("Batch size", 8, 64, 16)

# Load model
if 'generator' not in st.session_state:
    with st.spinner("Loading trained Generator..."):
        st.session_state.generator = load_generator()
st.success("✅ Generator loaded successfully!")

# Generate button
if st.button("🚀 Generate Synthetic Images", type="primary"):
    with st.spinner("Generating images..."):
        np.random.seed(seed)
        images = generate_images(
            st.session_state.generator,
            n_images,
            latent_dim=100,
            img_size=img_size,
            channels=1
        )

        # Display grid
        cols = st.columns(4)
        for i, img in enumerate(images):
            col_idx = i % 4
            with cols[col_idx]:
                st.image(img.squeeze(), caption=f"Synthetic #{i+1}", use_column_width=True)

        # Metrics
        diversity = np.var(images.reshape(n_images, -1), axis=0).mean()
        st.metric("Diversity Score", f"{diversity:.4f}")

        # Download options
        st.download_button(
            label="💾 Download ZIP",
            data=zip_images(images),
            file_name=f"synthetic_images_{datetime.now().strftime('%Y%m%d_%H%M')}.zip",
            mime="application/zip"
        )

def zip_images(images):
    """Convert images to ZIP bytes"""
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'w') as zf:
        for i, img in enumerate(images):
            img_pil = Image.fromarray((img.squeeze() * 255).astype(np.uint8))
            img_buffer = io.BytesIO()
            img_pil.save(img_buffer, format='PNG')
            zf.writestr(f"synthetic_{i+1:03d}.png", img_buffer.getvalue())
    zip_buffer.seek(0)
    return zip_buffer.getvalue()

# Demo section
st.markdown("---")
st.header("📊 Demo: Latent Space Interpolation")
if st.button("Show Interpolation"):
    interp_images = interpolate_demo(st.session_state.generator)
    st.image(interp_images, caption="Latent space morphing", use_column_width=True)

def interpolate_demo(generator, steps=10):
    """Create smooth interpolation between two latent vectors"""
    z1 = tf.random.normal([1, 100])
    z2 = tf.random.normal([1, 100])
    images = []
    for alpha in np.linspace(0, 1, steps):
        z_interp = (1-alpha) * z1 + alpha * z2
        img = generator(z_interp, training=False)
        img = (img + 1) / 2
        images.append(img[0].numpy())
    return np.hstack(images)

# Footer
st.markdown("---")
st.markdown("""
*Built with Vanilla GAN for privacy-preserving data augmentation*
**Compliant with GDPR, HIPAA, DPDP Act** | No real data memorized
""")


In [ ]:
# Start Streamlit in background + ngrok tunnel
import subprocess
import threading
import time
import os

# Terminal 1: Start Streamlit
def run_streamlit():
    os.system("streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &")

# Terminal 2: ngrok tunnel
def start_ngrok():
    time.sleep(5)  # Wait for Streamlit
    os.system("./ngrok http 8501")

# Run both
threading.Thread(target=run_streamlit).start()
threading.Thread(target=start_ngrok).start()

print("🚀 Streamlit app starting... Check the ngrok URL below in 10-20 seconds:")
print("Public URL will appear automatically")


In [ ]:
    # Save final models - CRITICAL for deployment
g_final_path = os.path.join(CONFIG["checkpoints_dir"], "G_final.keras")
d_final_path = os.path.join(CONFIG["checkpoints_dir"], "D_final.keras")
gan.generator.save(g_final_path)
gan.discriminator.save(d_final_path)
print("✅ Saved final models: G_final.keras, D_final.keras")
print("Files in checkpoints:", os.listdir("checkpoints"))


In [ ]:
def show_generated_grid(generator, n=16):
    noise = tf.random.normal((n, CONFIG["latent_dim"]))
    preds = generator(noise, training=False)
    imgs = (preds + 1.0) / 2.0
    side = int(np.sqrt(n))

    fig, axes = plt.subplots(side, side, figsize=(8, 8))
    idx = 0
    for i in range(side):
        for j in range(side):
            ax = axes[i, j] if side > 1 else axes[j]
            img = imgs[idx]
            ax.imshow(img.numpy().squeeze(), cmap="gray")
            ax.axis("off")
            idx += 1
    plt.tight_layout()
    plt.show()

show_generated_grid(gan.generator, n=16)
print("✅ Visual evaluation complete")

In [ ]:
# Install ngrok for public URL
!pip install -q pyngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz -O ngrok.tgz
!tar xvzf ngrok.tgz


In [ ]:
!./ngrok config add-authtoken 37WrtcnwEjuVhG10CNAzfMajLAl_9Xa6wKDE7kt9tUZ8C3fY


In [ ]:
# Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml

In [ ]:
%%writefile app_fixed.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import io
import zipfile
from datetime import datetime
import os

st.set_page_config(layout="wide")
st.title("🔬 Synthetic Image Generator")
st.markdown("**Vanilla GAN - Privacy-Preserving Data Augmentation**")

# File check
st.sidebar.markdown("**Files in checkpoints/**")
st.sidebar.write(os.listdir("checkpoints"))

if not os.path.exists("checkpoints/G_final.keras"):
    st.error("❌ Run Cell 2 first to save G_final.keras")
    st.stop()

@st.cache_resource
def load_generator():
    return tf.keras.models.load_model("checkpoints/G_final.keras", safe_mode=False)

generator = load_generator()
st.success("✅ Generator loaded successfully!")

# Controls
col1, col2 = st.columns([1,1])
with col1:
    n_images = st.slider("Number of images", 1, 64, 16)
with col2:
    seed = st.slider("Random seed", 0, 9999, 42)

if st.button("🚀 GENERATE SYNTHETIC IMAGES", type="primary"):
    np.random.seed(seed)
    noise = tf.random.normal([n_images, 100])
    images = generator(noise, training=False)
    images = (images + 1) / 2.0

    # Show grid
    cols = st.columns(4)
    for i, img in enumerate(images):
        with cols[i % 4]:
            st.image(img.squeeze(), caption=f"Synthetic #{i+1}", use_column_width=True)

    # ZIP download
    buf = io.BytesIO()
    with zipfile.ZipFile(buf, 'w') as zf:
        for i, img in enumerate(images):
            pil_img = Image.fromarray((img.squeeze()*255).astype(np.uint8))
            pil_img.save(zf, format='PNG')
    buf.seek(0)
    st.download_button("💾 Download ZIP Dataset", buf.getvalue(),
                      f"synthetic_dataset_{datetime.now().strftime('%Y%m%d_%H%M')}.zip")


In [ ]:
!pkill -f streamlit
from pyngrok import ngrok
ngrok.kill()


In [ ]:
%%writefile app_working.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import io
import zipfile
from datetime import datetime
import os

# Page config
st.set_page_config(layout="wide", page_title="GAN Generator")

st.markdown("""
# 🔬 **Synthetic Image Generator**
*Vanilla GAN - Privacy-Preserving Data Augmentation*
""")

# Load generator with error handling
@st.cache_resource
def load_model():
    model_path = "checkpoints/G_final.keras"
    if not os.path.exists(model_path):
        st.error(f"❌ {model_path} not found")
        st.stop()
    return tf.keras.models.load_model(model_path)

generator = load_model()
st.success("✅ Generator loaded!")

# Controls
col1, col2 = st.columns(2)
n_images = col1.slider("Images", 1, 64, 16)
seed = col2.slider("Seed", 0, 9999, 42)

# Generate button
if st.button("🚀 GENERATE SYNTHETIC IMAGES", type="primary"):
    with st.spinner("Generating..."):
        np.random.seed(seed)
        noise = tf.random.normal([n_images, 100])
        images = generator(noise, training=False)
        images = (images + 1) / 2.0  # Denormalize [-1,1] -> [0,1]

        # Display images
        cols = st.columns(4)
        for i, img in enumerate(images):
            with cols[i % 4]:
                st.image(img.squeeze(), caption=f"Synth #{i+1}", use_column_width=True)

        # Download ZIP
        buf = io.BytesIO()
        with zipfile.ZipFile(buf, "w") as zf:
            for i, img in enumerate(images):
                pil_img = Image.fromarray((img.squeeze() * 255).astype(np.uint8))
                pil_img.save(zf, "PNG")
        buf.seek(0)
        st.download_button("💾 Download ZIP", buf, f"gan_synthetic_{datetime.now().strftime('%Y%m%d_%H%M')}.zip")

st.markdown("---")
st.markdown("*Compliant with GDPR, HIPAA, DPDP Act*")


In [ ]:
# QUICK TEST - Run this to verify generator works
generator = tf.keras.models.load_model("checkpoints/G_final.keras")
noise = tf.random.normal([4, 100])
test_images = generator(noise, training=False)
test_images = (test_images + 1) / 2.0

import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 2, figsize=(6,6))
for i, ax in enumerate(axes.flat):
    ax.imshow(test_images[i].numpy().squeeze(), cmap='gray') # Added .numpy() here
    ax.axis('off')
plt.tight_layout()
plt.show()
print("✅ GENERATOR WORKS!")

In [ ]:
!pkill -f streamlit
!pip install -q gradio
from pyngrok import ngrok
ngrok.kill()


In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import zipfile
import io
from datetime import datetime
import os

# Load generator globally (outside Gradio)
print("Loading generator...")
generator = tf.keras.models.load_model("checkpoints/G_final.keras")
print("✅ Generator loaded!")

def generate_images(n_images, seed):
    np.random.seed(seed)
    noise = tf.random.normal([n_images, 100])
    images = generator(noise, training=False)
    images = (images + 1) / 2.0

    # Convert to PIL for Gradio
    pil_images = []
    for img in images:
        pil_img = Image.fromarray((img.squeeze() * 255).astype(np.uint8))
        pil_images.append(pil_img)

    return pil_images

def create_zip(n_images, seed):
    np.random.seed(seed)
    noise = tf.random.normal([n_images, 100])
    images = generator(noise, training=False)
    images = (images + 1) / 2.0

    buf = io.BytesIO()
    with zipfile.ZipFile(buf, "w") as zf:
        for i, img in enumerate(images):
            pil_img = Image.fromarray((img.squeeze() * 255).astype(np.uint8))
            pil_img.save(zf, "PNG")
    buf.seek(0)
    return buf.getvalue()

# Gradio interface
with gr.Blocks(title="GAN Generator") as demo:
    gr.Markdown("# 🔬 Synthetic Image Generator")
    gr.Markdown("*Vanilla GAN - Privacy-Preserving Data Augmentation*")

    with gr.Row():
        n_slider = gr.Slider(1, 64, value=16, label="Number of images")
        seed_slider = gr.Slider(0, 9999, value=42, label="Random seed")

    generate_btn = gr.Button("🚀 Generate Images", variant="primary", size="lg")

    gallery = gr.Gallery(label="Synthetic Images")

    # Create a button to trigger the zip creation and a File component to receive the output
    download_zip_button = gr.Button("💾 Create and Download ZIP", variant="secondary")
    zip_download_output = gr.File(label="Download ZIP Dataset", file_count="single", interactive=False)

    generate_btn.click(
        generate_images,
        inputs=[n_slider, seed_slider],
        outputs=gallery
    )

    download_zip_button.click(
        create_zip,
        inputs=[n_slider, seed_slider],
        outputs=zip_download_output
    )

demo.launch(share=True)

In [ ]:
import matplotlib.pyplot as plt
import os
print("📁 PROJECT FILES:")
print("- Samples:", len(os.listdir("samples")), "images ✓")
print("- G_final.keras:", os.path.exists("checkpoints/G_final.keras"), "✓")

# Show latest training sample
files = sorted(os.listdir("samples"))
img = plt.imread(f"samples/{files[-1]}")
plt.figure(figsize=(10,8))
plt.imshow(img)
plt.title("✅ Training Complete - Latest Samples", fontsize=16, pad=20)
plt.axis('off')
plt.show()


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

generator = tf.keras.models.load_model("checkpoints/G_final.keras")

@widgets.interact(n=(1,36,1), seed=(0,999,1))
def demo(n=16, seed=42):
    np.random.seed(seed)
    noise = tf.random.normal([n, 100])
    images = generator(noise, training=False)
    images = (images + 1) / 2

    rows = (n+3)//4
    fig, axes = plt.subplots(rows, 4, figsize=(12,3*rows))
    axes = axes.flat if rows > 1 else [axes]

    for i in range(n):
        ax = axes[i]
        ax.imshow(images[i].numpy().squeeze(), cmap='gray')
        ax.set_title(f"Synth #{i+1}", fontsize=10)
        ax.axis('off')
    plt.suptitle(f"🚀 LIVE GENERATION (n={n}, seed={seed})", fontsize=16)
    plt.tight_layout()
    plt.show()
